In [91]:
from unidecode import unidecode 
import pandas as pd
import re
import json
from datetime import datetime
import math

In [92]:
input_csv = "D:/TRUNG/FOODY/Raw/Foody/Foody_TienGiang.csv"
input_json = "D:/TRUNG/Database/Tiền Giang/Mongo_TienGiang.json"
output_update_csv = "DATA/Update_Agoda_TienGiang.csv"
output_add_csv = "DATA/Add_Agoda_TienGiang.csv"
output_json = "DATA/Mongo_TienGiang_cleaned.json"

In [93]:
with open(input_json, "r", encoding='utf-8') as file:
     raw_data = file.read()
processed_data1 = re.sub(r"NumberLong\((\d+)\)", r"\1", raw_data)
processed_data2 = re.sub(r"NumberInt\((\d+)\)", r"\1", processed_data1)
processed_data3 = re.sub(r'ISODate\("(.*?)"\)', r'"\1"', processed_data2)   
data = json.loads(processed_data3)
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [94]:
extracted_data = []
for item in data:
    _id = item.get("_id")
    name = item.get("name")
    lng = item.get("location", {}).get("lng")
    lat = item.get("location", {}).get("lat")
    address_components = {comp["types"][0]: comp["searchName"] for comp in item.get("addressComponents", [])}
    housenumber = address_components.get("housenumber", None)
    street = address_components.get("street", None)
    admin_level_4 = address_components.get("admin_level_4", None)
    extracted_data.append({
        "_id": _id,
        "name": name,
        "lng": lng,
        "lat": lat,
        "housenumber_searchName": housenumber,
        "street_searchName": street,
        "admin_level_4_searchName": admin_level_4
    })
mongo_df = pd.DataFrame(extracted_data)
display(mongo_df)

,_id,name,lng,lat,housenumber_searchName,street_searchName,admin_level_4_searchName
0,5c89d4ae95f3d0aabc3df161,Quán Bún Gỏi Già,106.367585,10.358817,186,nguyen hue,phuong 1
1,5c89ce5495f3d0aabc3df038,PNJ Co.opmart Gò Công,106.673138,10.354572,None,tran cong tuong,phuong 5
2,5c89ce5595f3d0aabc3df098,Taiphat-Electric,106.344177,10.350831,85/7,le thi hong gam,phuong 6
3,5c89d4ae95f3d0aabc3df114,Đại Lý Thuốc Sát Trùng 5 2 (đ Thành Được),106.111050,10.428740,None,cao dang chiem,tan binh
4,5c88d681d2c05acd14848e5a,Kem Gelato,106.355214,10.367014,143,ap bac,phuong 5
...,...,...,...,...,...,...,...
51001,64f6ef082b6bd13295ef184a,Ngân Hàng TMCP Quân Đội PGD Cai Lậy,106.118672,10.405064,None,quoc lo 1,phuong 4
51002,64f6ef082b6bd13295ef184d,Ngân Hàng TMCP Quân Đội CN Tiền Giang,106.361728,10.356927,116,nam ky khoi nghia,phuong 1
51003,64f6ef092b6bd13295ef1863,Ngân Hàng TMCP Quân Đội PGD Mỹ Tho,106.385913,10.351610,None,dinh bo linh,phuong 9
51004,65e048cd14770401f73d580f,Trạm Xe Bus Bến Xe Gò Công,106.661925,10.359629,None,tran cong tuong,phuong 4


In [95]:
foody_df = pd.read_csv(input_csv)
display(foody_df)

,Name.1,Address,District,City,Latitude,Longitude,Working hours
0,Phở Thủ,47 Trần Hưng Đạo,Thành Phố Mỹ Tho,Tiền Giang,10.353951,106.356647,06:00 - 21:30
1,Trạm Dừng Chân Phương Trang,Quốc Lộ 1A,Huyện Cái Bè,Tiền Giang,10.359338,105.978588,Cả ngày
2,1992 Chilli Queen - Food & Drink - Nam Kỳ Khởi...,137 Nam Kỳ Khởi Nghĩa. P. 4,Thành Phố Mỹ Tho,Tiền Giang,10.357797,106.361752,08:00 - 22:00
3,Cơm Tấm Cầu Đôi - Ấp Bắc,256A Ấp Bắc,Thành Phố Mỹ Tho,Tiền Giang,10.379343,106.342222,05:30 - 20:00
4,Quán Nem Nướng Alaghi,"Nguyễn Huệ, P. 2",Thị Xã Gò Công,Tiền Giang,10.359449,106.677666,17:00 - 23:00
...,...,...,...,...,...,...,...
483,Bách Tùng Viên,171B Xóm Dầu,Thành Phố Mỹ Tho,Tiền Giang,10.363217,106.370637,08:00 - 23:00
484,2001 Coffee,"Quốc Lộ 1A, Ấp 6, X. An Phú",Thị Xã Cai Lậy,Tiền Giang,10.401454,106.048440,07:00 - 21:30
485,Tiểu Lan Phố Cafe,"24 Yersin, P. 4",Thành Phố Mỹ Tho,Tiền Giang,10.357618,106.361100,07:00 - 22:30
486,Blue Rose Coffee & Tea - Trần Hưng Đạo,"99 Trần Hưng Đạo, P. 5",Thành Phố Mỹ Tho,Tiền Giang,10.364209,106.358157,06:00 - 22:00


In [96]:
def normalize(text):
    return unidecode(str(text)).lower() if pd.notnull(text) else None
foody_df['Address'] = foody_df['Address'].fillna('') + (', ' + foody_df['District'].fillna('') if foody_df['District'].notna().any() else '') + (', ' + foody_df['City'].fillna('') if foody_df['City'].notna().any() else '')
foody_df['Normalize'] = foody_df['Address'].apply(normalize)
normalized_dict = {row['Normalize']: row for _, row in foody_df.iterrows()}
matched_data = []
for _, row in mongo_df.iterrows():
    housenumber = normalize(row['housenumber_searchName'])
    street = normalize(row['street_searchName'])
    admin_level_4 = normalize(row['admin_level_4_searchName'])
    
    if None in [housenumber, street, admin_level_4]:
        continue
    for normalized_address, address_row in normalized_dict.items():
        if all(key in normalized_address for key in [housenumber, street, admin_level_4]):
            matched_data.append({
                "_id": row['_id'],
                "name": row['name'],
                "housenumber_searchName": row['housenumber_searchName'],
                "street_searchName": row['street_searchName'],
                "admin_level_4_searchName": row['admin_level_4_searchName'],
                "lat": row['lat'],
                "long": row['lng'],
                "Name.1": address_row['Name.1'],
                "Address": address_row['Address'],
                "Latitude": address_row['Latitude'],
                "Longitude": address_row['Longitude'],
                "Working hours": address_row['Working hours']
            })
matched_df = pd.DataFrame(matched_data)
def haversine(lat1, lng1, lat2, lng2):
    R = 6371.0  
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lng2 - lng1)
    
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance_km = R * c
    distance_m = distance_km * 1000  
    return round(distance_m, 2)
if not matched_df.empty:
    matched_df['lat'] = matched_df['lat'].astype(float)
    matched_df['long'] = matched_df['long'].astype(float)
    matched_df['Latitude'] = matched_df['Latitude'].astype(float)
    matched_df['Longitude'] = matched_df['Longitude'].astype(float)
    matched_df['Distance'] = matched_df.apply(
        lambda row: haversine(row['lat'], row['long'], row['Latitude'], row['Longitude']), 
        axis=1
    )
else:
    print("matched_df is empty, no data to process.")


In [97]:
add_df = foody_df[~foody_df['Name.1'].isin(matched_df['Name.1'])]
add_df.to_csv(output_add_csv, index=False, header=True, encoding='utf-8')
matched_df.to_csv(output_update_csv, index=False, header=True, encoding='utf-8')
display(matched_df)
display(add_df)

,_id,name,housenumber_searchName,street_searchName,admin_level_4_searchName,lat,long,Name.1,Address,Latitude,Longitude,Working hours,Distance
0,60637dd078f6610549f39d9d,Vinaphone Cái Bè,3,nguyen van tham,cai be,10.335514,106.033334,Tha Ly Cafe,"3 Nguyễn Văn Thâm, Huyện Cái Bè, Tiền Giang",10.335195,106.033064,07:00 - 21:00,46.22
1,6099d066491c036d235189db,09 Đường Lê Văn Nghề,9,le van nghe,dao thanh,10.365260,106.360648,Xưa Lắc Xưa Lơ - Các Món Ăn Vặt,"290B Lê Văn Nghề, X. Đạo Thạnh, Thành Phố Mỹ T...",10.370087,106.366225,14:00 - 22:00,812.59
2,6099ea91491c036d2354cee5,44 Đường Nguyễn Minh Đường,44,nguyen minh duong,dao thanh,10.371098,106.364472,Xe Lam - Buffet Nướng & Lẩu,"44 Nguyễn Minh Đường, Xã Đạo Thạnh, Thành Phố ...",10.370994,106.364382,16:00 - 22:00,15.16
3,60a3942e0309c615cc1dc221,Quán Hường - Bánh canh Cua & Hủ Tiếu,202g,pham ngu lao,cai be,10.333897,106.032389,Quán Hường - Bánh canh Cua & Hủ Tiếu,"202G Phạm Ngũ Lão, TT. Cái Bè, Huyện Cái Bè, T...",10.333897,106.032389,06:00 - 20:00,0.00
4,60a3942d0309c615cc1dc135,King Land Coffee & Milk Tea - Quốc Lộ 62,230,nguyen van con,vinh binh,10.354296,106.578940,King Land Coffee & Milk Tea - Quốc Lộ 62,"230 Nguyễn Văn Côn, thị trấn Vĩnh Bình, Huyện ...",10.354296,106.578940,06:00 - 22:00,0.00
5,60a3942c0309c615cc1dbf1c,Xe Lam - Buffet Nướng & Lẩu,44,nguyen minh duong,dao thanh,10.370994,106.364382,Xe Lam - Buffet Nướng & Lẩu,"44 Nguyễn Minh Đường, Xã Đạo Thạnh, Thành Phố ...",10.370994,106.364382,16:00 - 22:00,0.00
6,60a3942e0309c615cc1dc206,Tha Ly Cafe,3,nguyen van tham,cai be,10.335195,106.033064,Tha Ly Cafe,"3 Nguyễn Văn Thâm, Huyện Cái Bè, Tiền Giang",10.335195,106.033064,07:00 - 21:00,0.00
7,60a716ecb4e1e72f187eb357,Quán Cà Phê ThaLy,3,nguyen van tham,cai be,10.335228,106.033085,Tha Ly Cafe,"3 Nguyễn Văn Thâm, Huyện Cái Bè, Tiền Giang",10.335195,106.033064,07:00 - 21:00,4.34
8,60a716ecb4e1e72f187eaff3,DNTN THƯƠNG MẠI - KỸ THUẬT CÔNG NGHỆ NHẤT TÂM,13,quoc lo 1a,an thai trung,10.318276,105.890322,Sky Coffee,"13 Quốc Lộ 1A, Tổ 1, Ấp 1, Xã An Thái Trung, H...",10.318271,105.890922,08:00 - 22:00,65.62
9,60a716ecb4e1e72f187eaff7,Cửa Hàng Điện Gia Dụng Hải Tấn Phát,1,quoc lo 1a,an thai trung,10.320037,105.890584,Sky Coffee,"13 Quốc Lộ 1A, Tổ 1, Ấp 1, Xã An Thái Trung, H...",10.318271,105.890922,08:00 - 22:00,199.86


,Name.1,Address,District,City,Latitude,Longitude,Working hours,Normalize
0,Phở Thủ,"47 Trần Hưng Đạo, Thành Phố Mỹ Tho, Tiền Giang",Thành Phố Mỹ Tho,Tiền Giang,10.353951,106.356647,06:00 - 21:30,"47 tran hung dao, thanh pho my tho, tien giang"
1,Trạm Dừng Chân Phương Trang,"Quốc Lộ 1A, Huyện Cái Bè, Tiền Giang",Huyện Cái Bè,Tiền Giang,10.359338,105.978588,Cả ngày,"quoc lo 1a, huyen cai be, tien giang"
2,1992 Chilli Queen - Food & Drink - Nam Kỳ Khởi...,"137 Nam Kỳ Khởi Nghĩa. P. 4, Thành Phố Mỹ Tho,...",Thành Phố Mỹ Tho,Tiền Giang,10.357797,106.361752,08:00 - 22:00,"137 nam ky khoi nghia. p. 4, thanh pho my tho,..."
3,Cơm Tấm Cầu Đôi - Ấp Bắc,"256A Ấp Bắc, Thành Phố Mỹ Tho, Tiền Giang",Thành Phố Mỹ Tho,Tiền Giang,10.379343,106.342222,05:30 - 20:00,"256a ap bac, thanh pho my tho, tien giang"
4,Quán Nem Nướng Alaghi,"Nguyễn Huệ, P. 2, Thị Xã Gò Công, Tiền Giang",Thị Xã Gò Công,Tiền Giang,10.359449,106.677666,17:00 - 23:00,"nguyen hue, p. 2, thi xa go cong, tien giang"
...,...,...,...,...,...,...,...,...
483,Bách Tùng Viên,"171B Xóm Dầu, Thành Phố Mỹ Tho, Tiền Giang",Thành Phố Mỹ Tho,Tiền Giang,10.363217,106.370637,08:00 - 23:00,"171b xom dau, thanh pho my tho, tien giang"
484,2001 Coffee,"Quốc Lộ 1A, Ấp 6, X. An Phú, Thị Xã Cai Lậy, T...",Thị Xã Cai Lậy,Tiền Giang,10.401454,106.048440,07:00 - 21:30,"quoc lo 1a, ap 6, x. an phu, thi xa cai lay, t..."
485,Tiểu Lan Phố Cafe,"24 Yersin, P. 4, Thành Phố Mỹ Tho, Tiền Giang",Thành Phố Mỹ Tho,Tiền Giang,10.357618,106.361100,07:00 - 22:30,"24 yersin, p. 4, thanh pho my tho, tien giang"
486,Blue Rose Coffee & Tea - Trần Hưng Đạo,"99 Trần Hưng Đạo, P. 5, Thành Phố Mỹ Tho, Tiền...",Thành Phố Mỹ Tho,Tiền Giang,10.364209,106.358157,06:00 - 22:00,"99 tran hung dao, p. 5, thanh pho my tho, tien..."
